In [2]:
import numpy as np
import torch
import sys
sys.path.append('../')
from voting_games.werewolf_env_v0 import pare, Roles, Phase
from notebooks.learning_agents.models import ActorCriticAgent
from notebooks.learning_agents.utils import play_static_game, play_recurrent_game
from notebooks.learning_agents.static_agents import (
    random_approval_villager, 
    random_coordinated_approval_villager, 
    random_agent,
    random_approval_wolf,
    revenge_approval_wolf,
    coordinated_revenge_approval_wolf,
    random_likes_approval_wolf,
    aggressive_approval_wolf,
    )
import notebooks.learning_agents.stats as indicators
import random
import copy
from matplotlib import pyplot as plt
from tqdm import tqdm
from tabulate import tabulate

### Oracle Approval Voting

To test the theory that agents have learned how somewhat trust eachother based on likes and neutrals, we introduce a single oracle villager that likes every villager and dislikes every werewolf every single turn, except for voting rounds, where they abstain from voting altogether.

The thought process behind this is that even though villagers lose a valuable vote, the oracle's voting pattern will overcome this

In [5]:
env = pare(num_agents=10, werewolves=2, num_accusations=2)
observations, _, _, _, _ = env.reset()
obs_size= env.convert_obs(observations['player_0']['observation']).shape[-1]

trained_approval_agent = ActorCriticAgent({"rec_hidden_size": 256,
                                        "rec_layers": 1, 
                                        "joint_mlp_size": 128,
                                        "split_mlp_size": 128,
                                        "num_votes": 10,
                                        "approval_states": 3},
                                        num_players=10,
                                        obs_size=obs_size)
trained_approval_agent.load_state_dict(torch.load("../notebooks/stored_agents/lstm_first_no_one_hot_256_128/approval_agent_10_score_53.pth"))

<All keys matched successfully>

In [14]:
@torch.no_grad()
def play_recurrent_game_with_oracle(env, wolf_policy, villager_agent, num_times=10, hidden_state_size=None, voting_type=None, static_villager_policy="oracle", vtc_other_villagers=0, p=0.5):

    wins = 0
    game_replays = []
    oracle_ids = []

    for _ in range(num_times):
        next_observations, _, _, _, _ = env.reset()
        # init recurrent stuff for actor and critic to 0 as well
        static_villager = set([random.choice(list(set(env.agents) & set(env.world_state["villagers"])))])
        oracle_ids.append(list(static_villager)[-1])

        if static_villager_policy=="oracle":
            env.set_oracle(int(list(static_villager)[0].split("_")[-1]))

        magent_obs = {agent: {'obs': [], 
                              # obs size, and 1,1,64 as we pass batch first
                              'hcxs': [(torch.zeros((1,1,hidden_state_size), dtype=torch.float32), 
                                        torch.zeros((1,1,hidden_state_size), dtype=torch.float32))],
                    } for agent in env.agents if not env.agent_roles[agent]}

        wolf_action = None

        while env.agents:
            observations = copy.deepcopy(next_observations)
            actions = {}

            villagers = set(env.agents) & set(env.world_state["villagers"]) - static_villager
            wolves = set(env.agents) & set(env.world_state["werewolves"])

            ## VILLAGER LOGIC ##
            v_obs = torch.cat([torch.unsqueeze(torch.tensor(env.convert_obs(observations[villager]['observation']), dtype=torch.float), 0) for villager in villagers])

            # TODO: maybe this can be sped up? 
            hxs, cxs = zip(*[(hxs, cxs) for hxs, cxs in [magent_obs[villager]["hcxs"][-1] for villager in villagers]])
            hxs = torch.swapaxes(torch.cat(hxs),0,1)
            cxs = torch.swapaxes(torch.cat(cxs),0,1)

            policies, _ , cells = villager_agent(v_obs, (hxs, cxs))
            v_actions = torch.stack([p.sample() for p in policies], dim=1)

            hxs_new, cxs_new = cells
            hxs_new = torch.swapaxes(hxs_new,1,0)
            cxs_new = torch.swapaxes(cxs_new,1,0)

            for i, villager in enumerate(villagers):
                if voting_type == "plurality":
                    actions[villager] = v_actions[i].item()
                elif voting_type == "approval":
                    actions[villager] = (v_actions[i] - 1).tolist()
                magent_obs[villager]['hcxs'].append((torch.unsqueeze(hxs_new[i], 0), torch.unsqueeze(cxs_new[i], 0)))

            # if oracle is still alive
            if set(env.agents) & static_villager:
                static_villager_id = list(static_villager)[0]

                # get mode of the villager votes
                # max(lst, key=lst.count)

                if static_villager_policy == "oracle":

                    # vtc_other_villagers : what should the oracle do towards other villagers.
                    actions[static_villager_id] = [(-1 if random.random() < p else 0) if player in env.world_state['werewolves'] else vtc_other_villagers for player in env.possible_agents]
                    #actions[static_villager_id] = [-1 if player in env.world_state['werewolves'] else 0 for player in env.possible_agents]
                else: 
                    actions[static_villager_id] = random_agent(env, static_villager_id, action=None)

            ## WOLF LOGIC ## 
            phase = env.world_state['phase']
            for wolf in wolves:
                wolf_action = wolf_policy(env, wolf, action=wolf_action)
                actions[wolf] = wolf_action

            next_observations, _, _, _, _ = env.step(actions)

            ## UPDATED WOLF VARIABLE FOR WOLVES THAT COORDINATE ## 
            if env.world_state['phase'] == Phase.NIGHT:
                wolf_action = None
            
            if env.world_state['phase'] == Phase.ACCUSATION and phase == Phase.NIGHT:
                wolf_action = None
            
        ## Fill bigger buffer, keeping in mind sequence
        winner = env.world_state['winners']
        if winner == Roles.VILLAGER:
            wins += 1

        game_replays.append(copy.deepcopy(env.history))
    
    return wins, game_replays, oracle_ids

In [1]:
@torch.no_grad()
def play_recurrent_game_with_oracle(env, wolf_policy, villager_agent, num_times=10, hidden_state_size=None, voting_type=None, static_villager_policy="oracle", vtc_other_villagers=0, p=0.5):

    wins = 0
    game_replays = []
    oracle_ids = []

    for _ in range(num_times):
        next_observations, _, _, _, _ = env.reset()
        # init recurrent stuff for actor and critic to 0 as well
        static_villager = set([random.choice(list(set(env.agents) & set(env.world_state["villagers"])))])
        oracle_ids.append(list(static_villager)[-1])

        if static_villager_policy=="oracle":
            env.set_oracle(int(list(static_villager)[0].split("_")[-1]))

        magent_obs = {agent: {'obs': [], 
                              # obs size, and 1,1,64 as we pass batch first
                              'hcxs': [(torch.zeros((1,1,hidden_state_size), dtype=torch.float32), 
                                        torch.zeros((1,1,hidden_state_size), dtype=torch.float32))],
                    } for agent in env.agents if not env.agent_roles[agent]}

        wolf_action = None

        while env.agents:
            observations = copy.deepcopy(next_observations)
            actions = {}

            villagers = set(env.agents) & set(env.world_state["villagers"]) - static_villager
            wolves = set(env.agents) & set(env.world_state["werewolves"])

            ## VILLAGER LOGIC ##
            v_obs = torch.cat([torch.unsqueeze(torch.tensor(env.convert_obs(observations[villager]['observation']), dtype=torch.float), 0) for villager in villagers])

            # TODO: maybe this can be sped up? 
            hxs, cxs = zip(*[(hxs, cxs) for hxs, cxs in [magent_obs[villager]["hcxs"][-1] for villager in villagers]])
            hxs = torch.swapaxes(torch.cat(hxs),0,1)
            cxs = torch.swapaxes(torch.cat(cxs),0,1)

            policies, _ , cells = villager_agent(v_obs, (hxs, cxs))
            v_actions = torch.stack([p.sample() for p in policies], dim=1)

            hxs_new, cxs_new = cells
            hxs_new = torch.swapaxes(hxs_new,1,0)
            cxs_new = torch.swapaxes(cxs_new,1,0)

            for i, villager in enumerate(villagers):
                if voting_type == "plurality":
                    actions[villager] = v_actions[i].item()
                elif voting_type == "approval":
                    actions[villager] = (v_actions[i] - 1).tolist()
                magent_obs[villager]['hcxs'].append((torch.unsqueeze(hxs_new[i], 0), torch.unsqueeze(cxs_new[i], 0)))

            # if oracle is still alive
            if set(env.agents) & static_villager:
                static_villager_id = list(static_villager)[0]

                # get mode of the villager votes
                # max(lst, key=lst.count)

                if static_villager_policy == "oracle":

                    # vtc_other_villagers : what should the oracle do towards other villagers.
                    actions[static_villager_id] = [(-1 if random.random() < p else 0) if player in env.world_state['werewolves'] else vtc_other_villagers for player in env.possible_agents]
                    #actions[static_villager_id] = [-1 if player in env.world_state['werewolves'] else 0 for player in env.possible_agents]
                else: 
                    actions[static_villager_id] = random_agent(env, static_villager_id, action=None)

            ## WOLF LOGIC ## 
            phase = env.world_state['phase']
            for wolf in wolves:
                wolf_action = wolf_policy(env, wolf, action=wolf_action)
                actions[wolf] = wolf_action

            next_observations, _, _, _, _ = env.step(actions)

            ## UPDATED WOLF VARIABLE FOR WOLVES THAT COORDINATE ## 
            if env.world_state['phase'] == Phase.NIGHT:
                wolf_action = None
            
            if env.world_state['phase'] == Phase.ACCUSATION and phase == Phase.NIGHT:
                wolf_action = None
            
        ## Fill bigger buffer, keeping in mind sequence
        winner = env.world_state['winners']
        if winner == Roles.VILLAGER:
            wins += 1

        game_replays.append(copy.deepcopy(env.history))
    
    return wins, game_replays, oracle_ids

num_times = 1000
wins, replays = play_recurrent_game(env, random_approval_wolf, trained_approval_agent, num_times=num_times, hidden_state_size=256, voting_type="approval")
print(f'Without Oracle wins : {wins/float(num_times):.2f}')

def get_w_vote(p):
    return -1 if random.random() < p else 0

num_times=1000
vill_vote = [1, 0]
p = [0.0,0.25,0.5,0.75,1.0]
oracle_res = []
for how_v in vill_vote:
    for percent in p:
        wins, replays = play_recurrent_game_with_oracle(env, 
                                                        random_approval_wolf, 
                                                        trained_approval_agent, 
                                                        num_times=num_times, 
                                                        hidden_state_size=256, 
                                                        voting_type="approval", 
                                                        static_villager_policy="oracle",
                                                        vtc_other_villagers=how_v,
                                                        p=percent)
        print(f'Oracle wins with {how_v} for villagers and a chance of voting for werewolves at {percent}: {wins/float(num_times):.2f}')
        oracle_res.append(f'{wins/float(num_times):.2f}')
    print("\n")
#wins, replays = play_recurrent_game_with_oracle(env, random_approval_wolf, trained_approval_agent, num_times=num_times, hidden_state_size=256, voting_type="approval", static_villager_policy="oracle")
# print(f'With Oracle wins : {wins/float(num_times):.2f}')

# num_times = 1000
# wins, replays = play_recurrent_game_with_oracle(env, random_approval_wolf, trained_approval_agent, num_times=num_times, hidden_state_size=256, voting_type="approval", static_villager_policy="random")
# print(f'With other random villager wins : {wins/float(num_times):.2f}')


NameError: name 'torch' is not defined

In [15]:
# likelihood that oracle gets killed round 1, 2, 3, 4
# villager gets killed round 1, 2, 3, 4

wins, replays, o_ids = play_recurrent_game_with_oracle(env, 
                                                        random_approval_wolf, 
                                                        trained_approval_agent, 
                                                        num_times=100, 
                                                        hidden_state_size=256, 
                                                        voting_type="approval", 
                                                        static_villager_policy="oracle",
                                                        vtc_other_villagers=1,
                                                        p=0.0)


In [18]:
len(replays)

100

In [45]:
def _when_did_oracle_get_killed(game, id):
    '''
        What day and was he killed by votes or by werewolves
    '''
    just_votes = []
    just_kills = []
    for step in game:
        if step['phase'] == Phase.VOTING:
            if len(step["executed"]) == 1:
                if step['executed'][0] == id:
                    return step['day'], 0
            else:
                who_was_killed = list(set(step['executed']) - set(just_votes[-1]['executed']))[0]
                if who_was_killed == id:
                    return step['day'], 0
                
            just_votes.append(step)
        
        if step['phase'] == Phase.NIGHT:

            if len(step["killed"]) == 1:
                if step['killed'][0] == id:
                    return step['day'], 1
            else:
                who_was_killed = list(set(step['killed']) - set(just_kills[-1]['killed']))[0]
                if who_was_killed == id:
                    return step['day'], 1

            just_kills.append(step)              

    return -1, -1

# day, ty = [(_when_did_oracle_get_killed(replay, o_id)) for replay, o_id in zip(replays,o_ids)]
day, ty = list(zip(*[(_when_did_oracle_get_killed(replay, o_id)) for replay, o_id in zip(replays,o_ids)]))
game_winner = [replay[-1]['winners'] for replay in replays]

#print(f'Day - {day}, Type - {ty}, Winner - {replays[0][-1]["winners"]}')

In [49]:
import pandas as pd

df = pd.DataFrame({"day_oracle_killed": list(day), "way_oracle_killed": list(ty), "game_result": game_winner})
df.head()
df['day_oracle_killed'].value_counts()

-1    35
 3    21
 1    19
 2    17
 4     8
Name: day_oracle_killed, dtype: int64

In [50]:
df['way_oracle_killed'].value_counts()

-1    35
 0    33
 1    32
Name: way_oracle_killed, dtype: int64